In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
df = pd.read_csv("images.csv")
df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20


## Subject Focus

In [4]:
# Check the frequency of each category in the "Subject Focus" column
df["Subject Focus"].value_counts()

0    9638
1     274
Name: Subject Focus, dtype: int64

In [5]:
# Load the image data from the CSV file
data = pd.read_csv("images.csv")

# Define the image directory
image_dir = "images"

# Initialize lists to store images and labels
images = []
labels = []

# Define the desired image size
image_size = (64, 64)

# Iterate over the rows in the DataFrame
for index, row in data.iterrows():
    # Construct the image filename by appending the ".jpg" extension
    filename = row["Id"] + ".jpg"

    # Construct the path to the image file
    image_path = os.path.join(image_dir, filename)

    # Load and preprocess the image
    image = load_img(image_path, target_size=image_size)
    image = img_to_array(image) / 255.0

    # Add the preprocessed image to the images list
    images.append(image)

    # Extract the label from the "Subject Focus" column
    label = row["Subject Focus"]
    labels.append(label)

# Convert the images and labels lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Perform label encoding on the binary labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model1 = Sequential()
model1.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(32, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

# Compile the model with F1-score as the metric
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model1.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model1.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 10s 41ms/step - loss: 0.1509 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9710 - val_loss: 0.1044 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9778
Epoch 2/10
248/248 [==============================] - 10s 40ms/step - loss: 0.1305 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9710 - val_loss: 0.1083 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9778
Epoch 3/10
248/248 [==============================] - 10s 40ms/step - loss: 0.1188 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9710 - val_loss: 0.1018 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9778
Epoch 4/10
248/248 [==============================] - 14s 55ms/step - loss: 0.1146 - precision: 0.0000e+00 - recall: 0.0000e+00 - accuracy: 0.9710 - val_loss: 0.1037 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_accuracy: 0.9778
Epoch 5/10
248/248 [============

In [8]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9773071104387292
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
 [[1938    1]
 [  44    0]]


In [10]:
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

ROC AUC: 0.7324886305030709


In [6]:
# Save model architecture as JSON
model1_json = model1.to_json()
with open('model_focus.json', 'w') as json_file:
    json_file.write(model1_json)

# Save model weights
model1.save_weights('model_focus_weights.h5')

## Eyes

In [9]:
df["Eyes"].value_counts()

1    7658
0    2254
Name: Eyes, dtype: int64

In [7]:
labels = df["Eyes"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model2 = Sequential()
model2.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model2.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model2.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 11s 46ms/step - loss: 0.5465 - precision_1: 0.7701 - recall_1: 0.9961 - accuracy: 0.7679 - val_loss: 0.5212 - val_precision_1: 0.7827 - val_recall_1: 1.0000 - val_accuracy: 0.7827
Epoch 2/10
248/248 [==============================] - 12s 47ms/step - loss: 0.5357 - precision_1: 0.7701 - recall_1: 1.0000 - accuracy: 0.7701 - val_loss: 0.5380 - val_precision_1: 0.7827 - val_recall_1: 1.0000 - val_accuracy: 0.7827
Epoch 3/10
248/248 [==============================] - 11s 45ms/step - loss: 0.5287 - precision_1: 0.7702 - recall_1: 1.0000 - accuracy: 0.7702 - val_loss: 0.5192 - val_precision_1: 0.7827 - val_recall_1: 1.0000 - val_accuracy: 0.7827
Epoch 4/10
248/248 [==============================] - 11s 45ms/step - loss: 0.5194 - precision_1: 0.7709 - recall_1: 0.9985 - accuracy: 0.7703 - val_loss: 0.5152 - val_precision_1: 0.7824 - val_recall_1: 0.9987 - val_accuracy: 0.7816
Epoch 5/10
248/248 [==============================] - 12s 50ms/s

In [13]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.7307110438729199
Precision: 0.7962747380675204
Recall: 0.8814432989690721
F1-score: 0.836697247706422
Confusion Matrix:
 [[  81  350]
 [ 184 1368]]
ROC AUC: 0.5698238333293468


In [8]:
# Save model architecture as JSON
model2_json = model2.to_json()
with open('model_eye.json', 'w') as json_file:
    json_file.write(model2_json)

# Save model weights
model2.save_weights('model_eye_weights.h5')

## Face

In [14]:
df["Face"].value_counts()

1    8960
0     952
Name: Face, dtype: int64

In [9]:
labels = df["Face"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model3 = Sequential()
model3.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

# Compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model3.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model3.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 13s 53ms/step - loss: 0.3330 - precision_2: 0.9016 - recall_2: 0.9959 - accuracy: 0.8983 - val_loss: 0.3231 - val_precision_2: 0.9123 - val_recall_2: 1.0000 - val_accuracy: 0.9123
Epoch 2/10
248/248 [==============================] - 12s 49ms/step - loss: 0.3254 - precision_2: 0.9019 - recall_2: 1.0000 - accuracy: 0.9019 - val_loss: 0.2998 - val_precision_2: 0.9123 - val_recall_2: 1.0000 - val_accuracy: 0.9123
Epoch 3/10
248/248 [==============================] - 12s 48ms/step - loss: 0.3206 - precision_2: 0.9019 - recall_2: 1.0000 - accuracy: 0.9019 - val_loss: 0.2969 - val_precision_2: 0.9123 - val_recall_2: 1.0000 - val_accuracy: 0.9123
Epoch 4/10
248/248 [==============================] - 12s 50ms/step - loss: 0.3224 - precision_2: 0.9019 - recall_2: 1.0000 - accuracy: 0.9019 - val_loss: 0.2994 - val_precision_2: 0.9123 - val_recall_2: 1.0000 - val_accuracy: 0.9123
Epoch 5/10
248/248 [==============================] - 12s 49ms/s

In [16]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.8739283913262733
Precision: 0.9126521969295924
Recall: 0.953012714206744
F1-score: 0.932395889670092
Confusion Matrix:
 [[   9  165]
 [  85 1724]]
ROC AUC: 0.5531728331522464


In [10]:
# Save model architecture as JSON
model3_json = model3.to_json()
with open('model_face.json', 'w') as json_file:
    json_file.write(model3_json)

# Save model weights
model3.save_weights('model_face_weights.h5')

## Near

In [17]:
df["Near"].value_counts()

1    8540
0    1372
Name: Near, dtype: int64

In [14]:
labels = df["Near"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model4 = Sequential()
model4.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model4.add(MaxPooling2D((2, 2)))
model4.add(Conv2D(32, (3, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))
model4.add(Conv2D(64, (3, 3), activation='relu'))
model4.add(Flatten())
model4.add(Dense(64, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))

# Compile the model
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model4.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model4.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 10s 42ms/step - loss: 0.4083 - precision_4: 0.8645 - recall_4: 0.9958 - accuracy: 0.8614 - val_loss: 0.4072 - val_precision_4: 0.8492 - val_recall_4: 1.0000 - val_accuracy: 0.8492
Epoch 2/10
248/248 [==============================] - 10s 41ms/step - loss: 0.3733 - precision_4: 0.8647 - recall_4: 1.0000 - accuracy: 0.8647 - val_loss: 0.4021 - val_precision_4: 0.8492 - val_recall_4: 1.0000 - val_accuracy: 0.8492
Epoch 3/10
248/248 [==============================] - 10s 41ms/step - loss: 0.3541 - precision_4: 0.8647 - recall_4: 1.0000 - accuracy: 0.8647 - val_loss: 0.3813 - val_precision_4: 0.8492 - val_recall_4: 1.0000 - val_accuracy: 0.8492
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 0.3369 - precision_4: 0.8647 - recall_4: 1.0000 - accuracy: 0.8647 - val_loss: 0.3821 - val_precision_4: 0.8492 - val_recall_4: 1.0000 - val_accuracy: 0.8492
Epoch 5/10
248/248 [==============================] - 10s 40ms/s

In [19]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.8406454866364095
Precision: 0.8804226918798665
Recall: 0.9400237529691211
F1-score: 0.9092475588742103
Confusion Matrix:
 [[  84  215]
 [ 101 1583]]
ROC AUC: 0.7257405921559592


In [15]:
# Save model architecture as JSON
model4_json = model4.to_json()
with open('model_near.json', 'w') as json_file:
    json_file.write(model4_json)

# Save model weights
model4.save_weights('model_near_weights.h5')

## Action

In [20]:
df["Action"].value_counts()

0    9813
1      99
Name: Action, dtype: int64

In [16]:
labels = df["Action"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model5 = Sequential()
model5.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model5.add(MaxPooling2D((2, 2)))
model5.add(Conv2D(32, (3, 3), activation='relu'))
model5.add(MaxPooling2D((2, 2)))
model5.add(Conv2D(64, (3, 3), activation='relu'))
model5.add(Flatten())
model5.add(Dense(64, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))

# Compile the model
model5.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model5.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model5.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 12s 47ms/step - loss: 0.0725 - precision_5: 0.0370 - recall_5: 0.0128 - accuracy: 0.9870 - val_loss: 0.0601 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_accuracy: 0.9894
Epoch 2/10
248/248 [==============================] - 11s 44ms/step - loss: 0.0575 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - accuracy: 0.9902 - val_loss: 0.0587 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_accuracy: 0.9894
Epoch 3/10
248/248 [==============================] - 10s 42ms/step - loss: 0.0580 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - accuracy: 0.9902 - val_loss: 0.0583 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_accuracy: 0.9894
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 0.0556 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - accuracy: 0.9902 - val_loss: 0.0612 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00 - val_accuracy: 0.9894
Epoch 5/

In [22]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.989409984871407
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
 [[1962    0]
 [  21    0]]
ROC AUC: 0.5625212368331634


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Save model architecture as JSON
model5_json = model5.to_json()
with open('model_action.json', 'w') as json_file:
    json_file.write(model5_json)

# Save model weights
model5.save_weights('model_action_weights.h5')

## Accessory

In [23]:
df["Accessory"].value_counts()

0    9240
1     672
Name: Accessory, dtype: int64

In [18]:
labels = df["Accessory"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model6 = Sequential()
model6.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model6.add(MaxPooling2D((2, 2)))
model6.add(Conv2D(32, (3, 3), activation='relu'))
model6.add(MaxPooling2D((2, 2)))
model6.add(Conv2D(64, (3, 3), activation='relu'))
model6.add(Flatten())
model6.add(Dense(64, activation='relu'))
model6.add(Dense(1, activation='sigmoid'))

# Compile the model
model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model6.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model6.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 14s 55ms/step - loss: 0.2689 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - accuracy: 0.9285 - val_loss: 0.2639 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_accuracy: 0.9309
Epoch 2/10
248/248 [==============================] - 13s 51ms/step - loss: 0.2461 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - accuracy: 0.9325 - val_loss: 0.2380 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_accuracy: 0.9309
Epoch 3/10
248/248 [==============================] - 11s 46ms/step - loss: 0.2362 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - accuracy: 0.9325 - val_loss: 0.2256 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_accuracy: 0.9309
Epoch 4/10
248/248 [==============================] - 12s 49ms/step - loss: 0.2313 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - accuracy: 0.9325 - val_loss: 0.2272 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00 - val_accuracy: 0.9309


In [25]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.9268784669692385
Precision: 0.41304347826086957
Recall: 0.1386861313868613
F1-score: 0.20765027322404372
Confusion Matrix:
 [[1819   27]
 [ 118   19]]
ROC AUC: 0.6794489565127995


In [19]:
# Save model architecture as JSON
model6_json = model6.to_json()
with open('model_accessory.json', 'w') as json_file:
    json_file.write(model6_json)

# Save model weights
model6.save_weights('model_accessory_weights.h5')

## Group

In [26]:
df["Group"].value_counts()

0    8630
1    1282
Name: Group, dtype: int64

In [20]:
labels = df["Group"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model7 = Sequential()
model7.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model7.add(MaxPooling2D((2, 2)))
model7.add(Conv2D(32, (3, 3), activation='relu'))
model7.add(MaxPooling2D((2, 2)))
model7.add(Conv2D(64, (3, 3), activation='relu'))
model7.add(Flatten())
model7.add(Dense(64, activation='relu'))
model7.add(Dense(1, activation='sigmoid'))

# Compile the model
model7.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model7.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model7.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 12s 48ms/step - loss: 0.3959 - precision_7: 0.1562 - recall_7: 0.0049 - accuracy: 0.8692 - val_loss: 0.4004 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.8654
Epoch 2/10
248/248 [==============================] - 13s 52ms/step - loss: 0.3783 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - accuracy: 0.8720 - val_loss: 0.3966 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.8654
Epoch 3/10
248/248 [==============================] - 13s 51ms/step - loss: 0.3667 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - accuracy: 0.8720 - val_loss: 0.3803 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.8654
Epoch 4/10
248/248 [==============================] - 12s 49ms/step - loss: 0.3579 - precision_7: 0.0000e+00 - recall_7: 0.0000e+00 - accuracy: 0.8720 - val_loss: 0.3701 - val_precision_7: 0.0000e+00 - val_recall_7: 0.0000e+00 - val_accuracy: 0.8654
Epoch 5/

In [28]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.7831568330811901
Precision: 0.21403508771929824
Recall: 0.22846441947565543
F1-score: 0.22101449275362317
Confusion Matrix:
 [[1492  224]
 [ 206   61]]
ROC AUC: 0.6024637035872991


In [21]:
# Save model architecture as JSON
model7_json = model7.to_json()
with open('model_group.json', 'w') as json_file:
    json_file.write(model7_json)

# Save model weights
model7.save_weights('model_group_weights.h5')

## Collage

In [29]:
df["Collage"].value_counts()

0    9420
1     492
Name: Collage, dtype: int64

In [24]:
labels = df["Collage"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model8 = Sequential()
model8.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model8.add(MaxPooling2D((2, 2)))
model8.add(Conv2D(32, (3, 3), activation='relu'))
model8.add(MaxPooling2D((2, 2)))
model8.add(Conv2D(64, (3, 3), activation='relu'))
model8.add(Flatten())
model8.add(Dense(64, activation='relu'))
model8.add(Dense(1, activation='sigmoid'))

# Compile the model
model8.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model8.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model8.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 11s 43ms/step - loss: 0.1769 - precision_9: 0.0000e+00 - recall_9: 0.0000e+00 - accuracy: 0.9503 - val_loss: 0.1382 - val_precision_9: 0.0000e+00 - val_recall_9: 0.0000e+00 - val_accuracy: 0.9491
Epoch 2/10
248/248 [==============================] - 10s 39ms/step - loss: 0.1166 - precision_9: 0.8684 - recall_9: 0.0844 - accuracy: 0.9542 - val_loss: 0.1181 - val_precision_9: 0.7692 - val_recall_9: 0.3960 - val_accuracy: 0.9632
Epoch 3/10
248/248 [==============================] - 10s 42ms/step - loss: 0.0955 - precision_9: 0.8056 - recall_9: 0.5192 - accuracy: 0.9701 - val_loss: 0.1142 - val_precision_9: 0.7903 - val_recall_9: 0.4851 - val_accuracy: 0.9672
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 0.0762 - precision_9: 0.8435 - recall_9: 0.6343 - accuracy: 0.9762 - val_loss: 0.1205 - val_precision_9: 0.7079 - val_recall_9: 0.6238 - val_accuracy: 0.9677
Epoch 5/10
248/248 [============================

In [31]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.9687342410489158
Precision: 0.8305084745762712
Recall: 0.48514851485148514
F1-score: 0.6124999999999999
Confusion Matrix:
 [[1872   10]
 [  52   49]]
ROC AUC: 0.8790101114255953


In [26]:
# Save model architecture as JSON
model8_json = model8.to_json()
with open('model_collage.json', 'w') as json_file:
    json_file.write(model8_json)

# Save model weights
model8.save_weights('model_collage_weights.h5')

## Human

In [32]:
df["Human"].value_counts()

0    8264
1    1648
Name: Human, dtype: int64

In [27]:
labels = df["Human"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model9 = Sequential()
model9.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model9.add(MaxPooling2D((2, 2)))
model9.add(Conv2D(32, (3, 3), activation='relu'))
model9.add(MaxPooling2D((2, 2)))
model9.add(Conv2D(64, (3, 3), activation='relu'))
model9.add(Flatten())
model9.add(Dense(64, activation='relu'))
model9.add(Dense(1, activation='sigmoid'))

# Compile the model
model9.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model9.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model9.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 10s 41ms/step - loss: 0.4603 - precision_10: 0.0000e+00 - recall_10: 0.0000e+00 - accuracy: 0.8309 - val_loss: 0.4291 - val_precision_10: 0.0000e+00 - val_recall_10: 0.0000e+00 - val_accuracy: 0.8447
Epoch 2/10
248/248 [==============================] - 10s 42ms/step - loss: 0.4528 - precision_10: 0.0000e+00 - recall_10: 0.0000e+00 - accuracy: 0.8310 - val_loss: 0.4207 - val_precision_10: 0.0000e+00 - val_recall_10: 0.0000e+00 - val_accuracy: 0.8447
Epoch 3/10
248/248 [==============================] - 10s 41ms/step - loss: 0.4350 - precision_10: 0.0000e+00 - recall_10: 0.0000e+00 - accuracy: 0.8310 - val_loss: 0.4087 - val_precision_10: 0.0000e+00 - val_recall_10: 0.0000e+00 - val_accuracy: 0.8447
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 0.4199 - precision_10: 0.0000e+00 - recall_10: 0.0000e+00 - accuracy: 0.8310 - val_loss: 0.4120 - val_precision_10: 0.0000e+00 - val_recall_10: 0.0000e+00 - val_a

In [34]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.8320726172465961
Precision: 0.41935483870967744
Recall: 0.21103896103896103
F1-score: 0.28077753779697623
Confusion Matrix:
 [[1585   90]
 [ 243   65]]
ROC AUC: 0.6626070943981391


In [28]:
# Save model architecture as JSON
model9_json = model9.to_json()
with open('model_human.json', 'w') as json_file:
    json_file.write(model9_json)

# Save model weights
model9.save_weights('model_human_weights.h5')

## Occlusion

In [35]:
df["Occlusion"].value_counts()

0    8207
1    1705
Name: Occlusion, dtype: int64

In [29]:
labels = df["Occlusion"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model10 = Sequential()
model10.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model10.add(MaxPooling2D((2, 2)))
model10.add(Conv2D(32, (3, 3), activation='relu'))
model10.add(MaxPooling2D((2, 2)))
model10.add(Conv2D(64, (3, 3), activation='relu'))
model10.add(Flatten())
model10.add(Dense(64, activation='relu'))
model10.add(Dense(1, activation='sigmoid'))

# Compile the model
model10.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model10.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model10.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 10s 41ms/step - loss: 0.4614 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - accuracy: 0.8267 - val_loss: 0.4377 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00 - val_accuracy: 0.8331
Epoch 2/10
248/248 [==============================] - 10s 39ms/step - loss: 0.4402 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - accuracy: 0.8267 - val_loss: 0.4282 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00 - val_accuracy: 0.8331
Epoch 3/10
248/248 [==============================] - 10s 40ms/step - loss: 0.4269 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - accuracy: 0.8267 - val_loss: 0.4363 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00 - val_accuracy: 0.8331
Epoch 4/10
248/248 [==============================] - 10s 41ms/step - loss: 0.4087 - precision_11: 0.0000e+00 - recall_11: 0.0000e+00 - accuracy: 0.8267 - val_loss: 0.4130 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00 - val_a

In [37]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.8199697428139183
Precision: 0.4409090909090909
Recall: 0.2930513595166163
F1-score: 0.35208711433756806
Confusion Matrix:
 [[1529  123]
 [ 234   97]]
ROC AUC: 0.6818595787949058


In [30]:
# Save model architecture as JSON
model10_json = model10.to_json()
with open('model_occlusion.json', 'w') as json_file:
    json_file.write(model10_json)

# Save model weights
model10.save_weights('model_occlusion_weights.h5')

## Info

In [38]:
df["Info"].value_counts()

0    9305
1     607
Name: Info, dtype: int64

In [31]:
labels = df["Info"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model11 = Sequential()
model11.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model11.add(MaxPooling2D((2, 2)))
model11.add(Conv2D(32, (3, 3), activation='relu'))
model11.add(MaxPooling2D((2, 2)))
model11.add(Conv2D(64, (3, 3), activation='relu'))
model11.add(Flatten())
model11.add(Dense(64, activation='relu'))
model11.add(Dense(1, activation='sigmoid'))

# Compile the model
model11.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model11.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model11.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 10s 42ms/step - loss: 0.2428 - precision_12: 0.0312 - recall_12: 0.0021 - accuracy: 0.9356 - val_loss: 0.2033 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00 - val_accuracy: 0.9365
Epoch 2/10
248/248 [==============================] - 10s 41ms/step - loss: 0.1950 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - accuracy: 0.9393 - val_loss: 0.1781 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00 - val_accuracy: 0.9365
Epoch 3/10
248/248 [==============================] - 11s 45ms/step - loss: 0.1608 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - accuracy: 0.9393 - val_loss: 0.1524 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00 - val_accuracy: 0.9365
Epoch 4/10
248/248 [==============================] - 12s 48ms/step - loss: 0.1442 - precision_12: 0.7870 - recall_12: 0.1767 - accuracy: 0.9472 - val_loss: 0.1814 - val_precision_12: 0.5591 - val_recall_12: 0.5635 - val_accuracy: 0.9440
Epoch 5/

In [40]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.9536056480080686
Precision: 0.7023809523809523
Recall: 0.46825396825396826
F1-score: 0.5619047619047619
Confusion Matrix:
 [[1832   25]
 [  67   59]]
ROC AUC: 0.8497363045020558


In [32]:
# Save model architecture as JSON
model11_json = model11.to_json()
with open('model_info.json', 'w') as json_file:
    json_file.write(model11_json)

# Save model weights
model11.save_weights('model_info_weights.h5')

## Blur

In [41]:
df["Blur"].value_counts()

0    9214
1     698
Name: Blur, dtype: int64

In [12]:
labels = df["Blur"]
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create the CNN model
model12 = Sequential()
model12.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model12.add(MaxPooling2D((2, 2)))
model12.add(Conv2D(32, (3, 3), activation='relu'))
model12.add(MaxPooling2D((2, 2)))
model12.add(Conv2D(64, (3, 3), activation='relu'))
model12.add(Flatten())
model12.add(Dense(64, activation='relu'))
model12.add(Dense(1, activation='sigmoid'))

# Compile the model
model12.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), 'accuracy'])

# Train the model
model12.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# Make predictions on the test set
y_pred_prob = model12.predict(x_test)
y_pred = (y_pred_prob > 0.5).astype(int)

Epoch 1/10
248/248 [==============================] - 12s 47ms/step - loss: 0.2627 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - accuracy: 0.9265 - val_loss: 0.2211 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_accuracy: 0.9410
Epoch 2/10
248/248 [==============================] - 18s 72ms/step - loss: 0.2534 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - accuracy: 0.9267 - val_loss: 0.2163 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_accuracy: 0.9410
Epoch 3/10
248/248 [==============================] - 12s 49ms/step - loss: 0.2488 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - accuracy: 0.9267 - val_loss: 0.2164 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_accuracy: 0.9410
Epoch 4/10
248/248 [==============================] - 13s 52ms/step - loss: 0.2456 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - accuracy: 0.9267 - val_loss: 0.2298 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_accuracy: 0.9410


In [43]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", confusion_mat)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

Accuracy: 0.9409984871406959
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
 [[1866    0]
 [ 117    0]]
ROC AUC: 0.6574005368217587


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
# Save model architecture as JSON
model12_json = model12.to_json()
with open('model_blur.json', 'w') as json_file:
    json_file.write(model12_json)

# Save model weights
model12.save_weights('model_blur_weights.h5')